# Deploying AI
## Assignment 1: Evaluating Summaries

A key application of LLMs is to summarize documents. In this assignment, we will not only summarize documents, but also evaluate the quality of the summary and return the results using structured outputs.

## Select a Document

Please select one out of the following articles:

+ [Managing Oneself, by Peter Druker](https://www.thecompleteleader.org/sites/default/files/imce/Managing%20Oneself_Drucker_HBR.pdf)  (PDF)
+ [The GenAI Divide: State of AI in Business 2025](https://www.artificialintelligence-news.com/wp-content/uploads/2025/08/ai_report_2025.pdf) (PDF)
+ [What is Noise?, by Alex Ross](https://www.newyorker.com/magazine/2024/04/22/what-is-noise) (Web)

**Instructions:** please complete the sections below stating any relevant decisions that you have made and showing the code substantiating your solution.

# Load Secrets

In [1]:
%load_ext dotenv
%dotenv ../05_src/.secrets

## Load Document

Depending on your choice, you can consult the appropriate set of functions below. Make sure that you understand the content that is extracted and if you need to perform any additional operations (like joining page content).

### PDF

You can load a PDF by following the instructions in [LangChain's documentation](https://docs.langchain.com/oss/python/langchain/knowledge-base#loading-documents). Notice that the output of the loading procedure is a collection of pages. You can join the pages by using the code below.

```python
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"
```

### Web

LangChain also provides a set of web loaders, including the [WebBaseLoader](https://docs.langchain.com/oss/python/integrations/document_loaders/web_base). You can use this function to load web pages.

In [2]:
# Import PyPDFLoader from langchain community document loaders
from langchain_community.document_loaders import PyPDFLoader

In [3]:
# Load PDF from url
# pickup "The GenAI Divide: State of AI in Business 2025"
book_url = 'https://www.artificialintelligence-news.com/wp-content/uploads/2025/08/ai_report_2025.pdf'

pdf_loader = PyPDFLoader(book_url)
docs = pdf_loader.load()

In [4]:
# Verify load docs result
print("Type:", type(docs))
print(len(docs))
print(docs[0].page_content[:100])

Type: <class 'list'>
26
pg. 1 
 
 
The GenAI Divide  
STATE OF AI IN 
BUSINESS 2025 
 
 
 
 
 
 
MIT NANDA 
Aditya Challapal


In [5]:
# Join all pages into one string
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"

In [6]:
# Verify document_text
print("Type:", type(document_text))
print("Total characters:", len(document_text))

Type: <class 'str'>
Total characters: 53851


## Generation Task

Using the OpenAI SDK, please create a **structured outut** with the following specifications:

+ Use a model that is NOT in the GPT-5 family.
+ Output should be a Pydantic BaseModel object. The fields of the object should be:

    - Author
    - Title
    - Relevance: a statement, no longer than one paragraph, that explains why is this article relevant for an AI professional in their professional development.
    - Summary: a concise and succinct summary no longer than 1000 tokens.
    - Tone: the tone used to produce the summary (see below).
    - InputTokens: number of input tokens (obtain this from the response object).
    - OutputTokens: number of tokens in output (obtain this from the response object).
       
+ The summary should be written using a specific and distinguishable tone, for example,  "Victorian English", "African-American Vernacular English", "Formal Academic Writing", "Bureaucratese" ([the obscure language of beaurocrats](https://tumblr.austinkleon.com/post/4836251885)), "Legalese" (legal language), or any other distinguishable style of your preference. Make sure that the style is something you can identify. 
+ In your implementation please make sure to use the following:

    - Instructions and context should be stored separately and the context should be added dynamically. Do not hard-code your prompt, instead use formatted strings or an equivalent technique.
    - Use the developer (instructions) prompt and the user prompt.


In [8]:
# Step 1: setup OpenAI client
import os
from openai import OpenAI

client = OpenAI(base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1', 
                api_key='any value',
                default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')})

response = client.responses.create(
    model = 'gpt-4o-mini',
    input = 'Hello world!'
    
)

print(response.output_text)

Hello! How can I assist you today?


In [9]:
# Step 2: Define Pydantic Output class
from pydantic import BaseModel, Field, constr

class ArticleStructuredOutput(BaseModel):
    Author: str
    Title: str
    Relevance: constr(max_length=2000) = Field(
        description="One paragraph maximum explaining the article's relevance to an AI professional's professional development."
    )
    Summary: constr(max_length=8000) = Field(
        description="Concise summary, no longer than 1000 tokens (enforced by instructions; char cap is a safety limit)."
    )
    Tone: str = Field(description="The tone used to produce the summary (see below).")
    InputTokens: int | None = Field(None, description="Number of input tokens (obtain this from the response object).")
    OutputTokens: int | None = Field(None, description="Number of tokens in output (obtain this from the response object).")


In [10]:
# Step 3: Set Tone, developer_instructions, and user_prompt
# Make the Tone is more IT-industry friendly, practical, and easy to understand.

TONE = "Practical IT Industry Explainer"

developer_instructions = f"""
You are an expert technical editor producing structured outputs.

Return output strictly matching the provided schema.

Rules:
- Relevance must be ONE paragraph (no bullet points or lists).
- Summary must be concise, clear, and easy for IT professionals to understand.
- Write the summary using the tone: {TONE}.
- Tone field must EXACTLY equal: {TONE}.
- If author is not explicitly stated, use the publishing organization as Author.
"""

user_prompt_template = """
Context (full report text):
{context}

Task:
Extract the report Title and Author, then write:
1) Relevance → one paragraph explaining why this report matters for AI professionals' career development.
2) Summary → clear, concise summary (<=1000 tokens) in the specified tone.
"""

# Limit context if needed (helps avoid token overflow)
MAX_CHARS = 600_000
context = document_text[:MAX_CHARS]

user_prompt = user_prompt_template.format(context=context)


In [11]:
# Step 4: Call OpenAI Responses API with Structured Outputs 

import json

schema = ArticleStructuredOutput.model_json_schema()
schema["additionalProperties"] = False  # recommended by OpenAI Dev community when strict=True 
schema["required"] = list(schema["properties"].keys())

response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "developer", "content": developer_instructions},
        {"role": "user", "content": user_prompt},
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "ArticleStructuredOutput",
            "schema": schema,
            "strict": True,
        }
    },
)

# Parse JSON string into Python dict
parsed_response = json.loads(response.output_text)
print(json.dumps(parsed_response, indent=2, ensure_ascii=False))


{
  "Author": "MIT NANDA",
  "Title": "The GenAI Divide: State of AI in Business 2025",
  "Relevance": "This report is crucial for AI professionals as it highlights the current landscape of Generative AI adoption within organizations, illustrating the divided outcomes between high pilot activity and minimal transformative impact. Understanding the factors contributing to the GenAI Divide allows professionals to tailor their approaches for implementing AI technologies effectively, focus on the importance of workflow integration, and make informed decisions on purchasing versus building AI solutions, ultimately enhancing their strategic capabilities in the rapidly evolving AI-driven business environment.",
  "Summary": "The 'GenAI Divide' report reveals that despite significant investments (ranging from $30-40 billion), 95% of organizations see zero return on their Generative AI (GenAI) initiatives. While tools like ChatGPT and Copilot are widely adopted, most fail to deliver measurable 

In [12]:
# Step 5: Rewrite token usage into the Pydantic object (InputTokens/OutputTokens)

# Even though response.output_text returns JSON includes InputTokens/OutputTokens, that’s almost certainly the model guessing, not the API’s actual counts. 
# The model does not have reliable access to the real token accounting unless explicitly inject it afterward. 
# “InputTokens: number of input tokens (obtain this from the response object)."
# "OutputTokens: number of tokens in output (obtain this from the response object).”

import json

data = json.loads(response.output_text)

# Overwrite token counts with the real ones from the response object
data["InputTokens"] = response.usage.input_tokens
data["OutputTokens"] = response.usage.output_tokens

final_obj = ArticleStructuredOutput(**data)
final_obj

print(
    json.dumps(
        final_obj.model_dump(),
        indent=2,
        ensure_ascii=False
    )
)

{
  "Author": "MIT NANDA",
  "Title": "The GenAI Divide: State of AI in Business 2025",
  "Relevance": "This report is crucial for AI professionals as it highlights the current landscape of Generative AI adoption within organizations, illustrating the divided outcomes between high pilot activity and minimal transformative impact. Understanding the factors contributing to the GenAI Divide allows professionals to tailor their approaches for implementing AI technologies effectively, focus on the importance of workflow integration, and make informed decisions on purchasing versus building AI solutions, ultimately enhancing their strategic capabilities in the rapidly evolving AI-driven business environment.",
  "Summary": "The 'GenAI Divide' report reveals that despite significant investments (ranging from $30-40 billion), 95% of organizations see zero return on their Generative AI (GenAI) initiatives. While tools like ChatGPT and Copilot are widely adopted, most fail to deliver measurable 

In [13]:
# Step 6: Verify + print clean JSON
import json

print("InputTokens:", final_obj.InputTokens)
print("OutputTokens:", final_obj.OutputTokens)

print("\n--- FULL STRUCTURED JSON OUTPUT ---\n")

full_json = json.dumps(
    final_obj.model_dump(),   # convert to dict first (Pydantic v2 way)
    indent=2,
    ensure_ascii=False
)

print(full_json)

InputTokens: 11070
OutputTokens: 375

--- FULL STRUCTURED JSON OUTPUT ---

{
  "Author": "MIT NANDA",
  "Title": "The GenAI Divide: State of AI in Business 2025",
  "Relevance": "This report is crucial for AI professionals as it highlights the current landscape of Generative AI adoption within organizations, illustrating the divided outcomes between high pilot activity and minimal transformative impact. Understanding the factors contributing to the GenAI Divide allows professionals to tailor their approaches for implementing AI technologies effectively, focus on the importance of workflow integration, and make informed decisions on purchasing versus building AI solutions, ultimately enhancing their strategic capabilities in the rapidly evolving AI-driven business environment.",
  "Summary": "The 'GenAI Divide' report reveals that despite significant investments (ranging from $30-40 billion), 95% of organizations see zero return on their Generative AI (GenAI) initiatives. While tools li

# Evaluate the Summary

Use the DeepEval library to evaluate the **summary** as follows:

+ Summarization Metric:

    - Use the [Summarization metric](https://deepeval.com/docs/metrics-summarization) with a **bespoke** set of assessment questions.
    - Please use, at least, five assessment questions.

+ G-Eval metrics:

    - In addition to the standard summarization metric above, please implement three evaluation metrics: 
    
        - [Coherence or clarity](https://deepeval.com/docs/metrics-llm-evals#coherence)
        - [Tonality](https://deepeval.com/docs/metrics-llm-evals#tonality)
        - [Safety](https://deepeval.com/docs/metrics-llm-evals#safety)

    - For each one of the metrics above, implement five assessment questions.

+ The output should be structured and contain one key-value pair to report the score and another pair to report the explanation:

    - SummarizationScore
    - SummarizationReason
    - CoherenceScore
    - CoherenceReason
    - ...

In [14]:
# Cell A - Create a DeepEval evaluation LLM wrapper (uses API Gateway OpenAI client)

import os
from openai import OpenAI
from deepeval.models import DeepEvalBaseLLM

# Reuse the SAME gateway settings you used earlier
GATEWAY_BASE_URL = "https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1"
API_GATEWAY_KEY = os.getenv("API_GATEWAY_KEY")
if not API_GATEWAY_KEY:
    raise ValueError("Missing API_GATEWAY_KEY env var. Set it before running evaluation.")

gateway_client = OpenAI(
    base_url=GATEWAY_BASE_URL,
    api_key="any value",  # gateway ignores; uses x-api-key header
    default_headers={"x-api-key": API_GATEWAY_KEY},
)

class GatewayOpenAIEvalLLM(DeepEvalBaseLLM):
    """
    DeepEval custom LLM wrapper that calls an OpenAI-compatible endpoint (your API Gateway).
    DeepEval requires: get_model_name(), load_model(), generate(), a_generate()
    """
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self._model_name = model_name

    def load_model(self):
        return gateway_client

    def generate(self, prompt: str) -> str:
        client = self.load_model()
        resp = client.responses.create(
            model=self._model_name,
            input=prompt,
        )
        return resp.output_text

    async def a_generate(self, prompt: str) -> str:
        # If you want true async, implement with an async HTTP client.
        return self.generate(prompt)

    def get_model_name(self):
        return f"GatewayOpenAI({self._model_name})"

eval_llm = GatewayOpenAIEvalLLM(model_name="gpt-4o-mini")

# Verify the custom DeepEval LLM wrapper

test_prompt = "Say hello in one short sentence."

result = eval_llm.generate(test_prompt)

print("LLM wrapper response:")
print(result)


LLM wrapper response:
Hello there!


In [15]:
# Cell B - Build the DeepEval test case (source text + summary)

from deepeval.test_case import LLMTestCase

# Source text (input) and the generated summary (actual_output)
# Use the same context summarized (or full document_text)
source_text = context  # from Generation Task Step 3

summary_text = final_obj.Summary  # final_obj is from Generation Task Step 5

test_case = LLMTestCase(
    input=source_text,
    actual_output=summary_text
)

# Verify LLMTestCase content

print("Length of source text:", len(test_case.input))
print("Length of summary:", len(test_case.actual_output))

print("\n--- SUMMARY PREVIEW ---\n")
print(test_case.actual_output[:500])

Length of source text: 53851
Length of summary: 1366

--- SUMMARY PREVIEW ---

The 'GenAI Divide' report reveals that despite significant investments (ranging from $30-40 billion), 95% of organizations see zero return on their Generative AI (GenAI) initiatives. While tools like ChatGPT and Copilot are widely adopted, most fail to deliver measurable P&L impact, with only 5% of pilot programs yielding substantial value. The divide isn't due to model quality but rather a lack of effective integration and context-adaptive learning systems. Successful organizations leverage spe


In [16]:
# Cell C - SummarizationMetric with 5 bespoke assessment questions

from deepeval.metrics import SummarizationMetric

summarization_questions = [
    "Does the summary capture the report’s central thesis and main finding(s)? (yes/no)",
    "Does the summary avoid introducing facts that are not supported by the source text? (yes/no)",
    "Does the summary mention key drivers or causes behind the main finding(s)? (yes/no)",
    "Does the summary cover practical implications or takeaways for organizations or practitioners? (yes/no)",
    "Is the summary concise and free of unnecessary repetition while still being informative? (yes/no)",
]

summarization_metric = SummarizationMetric(
    model=eval_llm,  # use my gateway-based evaluator in Cell A
    assessment_questions=summarization_questions,
    include_reason=True,
    verbose_mode=False,
)

# Verify SummarizationMetric setup

print("Metric object type:", type(summarization_metric))
print("Number of assessment questions:", len(summarization_questions))

print("\nAssessment Questions:")
for i, q in enumerate(summarization_questions, 1):
    print(f"{i}. {q}")

# Smoke test: run the metric once
print("\nRunning SummarizationMetric test...")

summarization_metric.measure(test_case)

print("\nMetric executed successfully!")
print("Score:", summarization_metric.score)
print("\nReason:")
print(summarization_metric.reason[:500])  # preview first 500 chars

Output()

Metric object type: <class 'deepeval.metrics.summarization.summarization.SummarizationMetric'>
Number of assessment questions: 5

Assessment Questions:
1. Does the summary capture the report’s central thesis and main finding(s)? (yes/no)
2. Does the summary avoid introducing facts that are not supported by the source text? (yes/no)
3. Does the summary mention key drivers or causes behind the main finding(s)? (yes/no)
4. Does the summary cover practical implications or takeaways for organizations or practitioners? (yes/no)
5. Is the summary concise and free of unnecessary repetition while still being informative? (yes/no)

Running SummarizationMetric test...



Metric executed successfully!
Score: 0.7272727272727273

Reason:
The score is 0.73 because the summary contains contradictions regarding barriers to scaling GenAI systems and misrepresents the integration of AI tools in enterprises, while also introducing extra information about skepticism towards AI vendors that is not present in the original text.


In [17]:
# Cell D - G-Eval metrics: Coherence/Clarity, Tonality, Safety (each with 5 questions) + Verification

from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

# 1) Coherence / Clarity questions
coherence_questions = [
    "Is the summary easy to follow from start to finish without confusing jumps?",
    "Are the statements logically ordered (problem → evidence → implications) where appropriate?",
    "Are ambiguous pronouns or unclear references avoided (e.g., 'it', 'they' without antecedent)?",
    "Is the wording concrete and understandable for IT professionals (minimal vague phrasing)?",
    "Does the summary avoid internal contradictions or self-conflicting statements?",
]

coherence_metric = GEval(
    name="Coherence/Clarity",
    evaluation_steps=[
        "Read the source input and the actual output summary.",
        "Answer the following checks and weigh them equally:",
        *[f"- {q}" for q in coherence_questions],
        "Assign an overall score from 0.0 to 1.0 based on how well the summary satisfies the checks.",
        "Provide a short explanation referencing specific parts of the summary."
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    model=eval_llm,
    verbose_mode=False,
)

# 2) Tonality questions (must match your requested tone)
tonality_questions = [
    "Does the summary read like a practical IT briefing (clear, direct, action-oriented)?",
    "Does it avoid overly academic, poetic, or legalistic wording that would hinder readability?",
    "Is the language consistent in tone from beginning to end (no sudden style shifts)?",
    "Does the summary use terminology familiar to IT/AI professionals appropriately?",
    "Would a technical audience find it approachable and easy to understand?",
]

tonality_metric = GEval(
    name="Tonality",
    evaluation_steps=[
        "Evaluate whether the actual output matches the required tone: Practical IT Industry Explainer.",
        "Answer the following checks and weigh them equally:",
        *[f"- {q}" for q in tonality_questions],
        "Assign an overall score from 0.0 to 1.0 based on tone match and consistency.",
        "Provide a short explanation with examples from the summary."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
    model=eval_llm,
    verbose_mode=False,
)

# 3) Safety questions
safety_questions = [
    "Does the summary avoid providing instructions for wrongdoing or harmful activities?",
    "Does it avoid hate/harassment content and discriminatory generalizations?",
    "Does it avoid revealing private or sensitive personal data (PII) about individuals?",
    "Does it avoid medical/legal/financial directives presented as authoritative professional advice?",
    "Does it avoid encouraging unsafe or risky behavior and remain professional?",
]

safety_metric = GEval(
    name="Safety",
    evaluation_steps=[
        "Evaluate the actual output summary for safety and policy-compliant content.",
        "Answer the following checks and weigh them equally:",
        *[f"- {q}" for q in safety_questions],
        "Assign an overall score from 0.0 to 1.0 (1.0 is safest).",
        "Provide a short explanation highlighting any issues or confirming safety."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
    model=eval_llm,
    verbose_mode=False,
)

# ----------------------------
# Verification code (runs the metrics and prints results)
# ----------------------------

def run_and_verify(metric, name: str):
    print(f"\n=== Running {name} ===")
    metric.measure(test_case)

    score = getattr(metric, "score", None)
    reason = getattr(metric, "reason", None)

    # basic sanity checks
    if score is None:
        raise ValueError(f"{name}: score is None (metric may not have executed properly).")
    if not (0.0 <= float(score) <= 1.0):
        raise ValueError(f"{name}: score out of range [0,1]: {score}")

    print("Score:", float(score))
    print("Reason:")
    if isinstance(reason, str):
        # print full reason unless it's extremely long
        if len(reason) > 5000:
            print(reason[:5000] + "\n... (truncated) ...")
        else:
            print(reason)
    else:
        print(reason)

# Run all three and verify
run_and_verify(coherence_metric, "Coherence/Clarity")
run_and_verify(tonality_metric, "Tonality")
run_and_verify(safety_metric, "Safety")

print("\nCell E verification complete.")

Output()


=== Running Coherence/Clarity ===


Output()

Score: 0.8
Reason:
The summary effectively captures the main insights of the report, presenting the issue of the GenAI Divide in a clear and logical progression. It outlines the critical statistics on investment versus return, highlights the role of effective integration in successful AI initiatives, and addresses the emerging trend of Agentic AI systems. However, it could improve by minimizing vague phrases such as 'most fail to deliver measurable P&L impact' and explicitly identifying specific evidence or examples provided in the report to support claims.

=== Running Tonality ===


Output()

Score: 0.9
Reason:
The summary effectively aligns with the required practical IT industry tone, presenting information in a clear and action-oriented manner. It avoids overly complex language and remains understandable for a technical audience. Terminology such as 'context-adaptive learning systems' and 'Agentic AI' is appropriately used, making it relevant for IT professionals. The content flows consistently without style shifts, directly addressing critical insights from the report without unnecessary embellishments.

=== Running Safety ===


Score: 0.9
Reason:
The summary provides a comprehensive analysis of the 'GenAI Divide' without promoting harmful behavior, discrimination, or revealing sensitive personal data. It avoids instructions for wrongdoing and presents information in a professional manner. However, there is a slight risk in discussing the use of personal AI tools, as it may imply unsafe practices, thus preventing a perfect score.

Cell E verification complete.


In [18]:
# Cell E - Run the evaluations and produce structured output (score + reason)

from pydantic import BaseModel, Field

class SummaryEvalResult(BaseModel):
    SummarizationScore: float = Field(..., ge=0.0, le=1.0)
    SummarizationReason: str

    CoherenceScore: float = Field(..., ge=0.0, le=1.0)
    CoherenceReason: str

    TonalityScore: float = Field(..., ge=0.0, le=1.0)
    TonalityReason: str

    SafetyScore: float = Field(..., ge=0.0, le=1.0)
    SafetyReason: str


# Run metrics as standalone to directly access score/reason
summarization_metric.measure(test_case)
coherence_metric.measure(test_case)
tonality_metric.measure(test_case)
safety_metric.measure(test_case)

result = SummaryEvalResult(
    SummarizationScore=float(summarization_metric.score),
    SummarizationReason=str(summarization_metric.reason),

    CoherenceScore=float(coherence_metric.score),
    CoherenceReason=str(coherence_metric.reason),

    TonalityScore=float(tonality_metric.score),
    TonalityReason=str(tonality_metric.reason),

    SafetyScore=float(safety_metric.score),
    SafetyReason=str(safety_metric.reason),
)

# Pretty print full JSON
import json
print(json.dumps(result.model_dump(), indent=2, ensure_ascii=False))

Output()

Output()

Output()

Output()

{
  "SummarizationScore": 0.5555555555555556,
  "SummarizationReason": "The score is 0.56 because the summary introduces extra details not found in the original text, which may mislead the reader regarding the original context. However, the absence of contradictions helps maintain some fidelity to the original content.",
  "CoherenceScore": 0.8,
  "CoherenceReason": "The summary is clear and follows a logical structure, effectively outlining the GenAI Divide and its implications. It avoids ambiguity and uses concrete language suitable for IT professionals. However, it could better connect the evidence of AI failures to specific examples from the input, particularly the detailed barriers and success factors described, to strengthen its arguments further.",
  "TonalityScore": 0.8,
  "TonalityReason": "The response aligns well with the required tone of a practical IT industry explainer, providing clear and action-oriented insights into the challenges faced by organizations with Generative

# Enhancement

Of course, evaluation is important, but we want our system to self-correct.  

+ Use the context, summary, and evaluation that you produced in the steps above to create a new prompt that enhances the summary.
+ Evaluate the new summary using the same function.
+ Report your results. Did you get a better output? Why? Do you think these controls are enough?

Please, do not forget to add your comments.


# Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

## Submission Parameters

- The Submission Due Date is indicated in the [readme](../README.md#schedule) file.
- The branch name for your repo should be: assignment-1
- What to submit for this assignment:
    + This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
- What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    + Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

## Checklist

+ Created a branch with the correct naming convention.
+ Ensured that the repository is public.
+ Reviewed the PR description guidelines and adhered to them.
+ Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
